In [26]:
from genshintoolkit.gtcommon.gtenum import *
from genshintoolkit.gtcommon.atfdata import *
from genshintoolkit.gtcommon.reaction import reaction_catalyze

In [27]:
multiplier_e=[
    ('atk',1.858),
    ('em',3.715),
]

dmgb_e=lambda em:(em-200)*0.001 if 200<=em<=1000 else (0.8 if em>1000 else 0)
cr_e=lambda em:(em-200)*0.0003 if 200<=em<=1000 else (0.24 if em>1000 else 0)

baseatk_nahida=299
em_nahida=115

em_weapon1=265
baseatk_weapon1=542

em_weapon1_effect=lambda charcount_same:charcount_same*32
dmgb_weapon1_effect=lambda charcount_diff: charcount_diff*0.1


In [28]:
def calc(atkp,em,cr,cd,grass_globlet=True):
    factor_def_res=0.5*(1+0.4/2)
    atk=(baseatk_nahida+baseatk_weapon1)*(1+atkp)+311
    em=em_nahida+em_weapon1+em_weapon1_effect(0)+em+187+80+150+32
    cr_bonus=cr_e(em)
    crit=0.05+cr+0.311
    cdmg=0.5+cd

    basedmg=atk*1.858+em*3.175+reaction_catalyze(EnumCatalyzeReaction.spread,em)*0.5

    return basedmg*(1+(0.466 if grass_globlet else 0)+dmgb_e(em)+dmgb_weapon1_effect(3))*(1+crit*cdmg)*factor_def_res,atk,em,crit,cdmg


In [31]:
from genshintoolkit.math_tools import get_point_distribution
from genshintoolkit.gtcommon.atfdata import substat_values_avg

distr=get_point_distribution(35,4)
max=[None,None]
for i in distr:
    atk,em,cr,cd=i
    if cr<30 and cd+cr<=30 and em<=30:
        result=calc(
            substat_values_avg[EnumStats.atk_percent]*atk,
            substat_values_avg[EnumStats.em]*em,
            substat_values_avg[EnumStats.crit_rate]*cr,
            substat_values_avg[EnumStats.crit_dmg]*cd
        )
        if max[0] is None:
            max[0]=result
        else:
            if result[0]>max[0][0]:
                max[0]=result
                max[1]=i

dmg,atk,em,cr,cd=max[0]
print(max)
print(f'最高期望{dmg:.2f}，攻击：{atk}，精通：{em}，暴击{cr:.2f}，爆伤：{cd:.2f}')




[(27210.84346315704, 1152.0, 929, 0.79, 1.622), (0, 5, 13, 17)]
最高期望27210.84，攻击：1152.0，精通：929，暴击0.79，爆伤：1.62


In [30]:
calc(
            substat_values_avg[EnumStats.atk_percent]*0,
            substat_values_avg[EnumStats.em]*22,
            substat_values_avg[EnumStats.crit_rate]*0,
            substat_values_avg[EnumStats.crit_dmg]*6
        )

(19149.27500471897, 1152.0, 1269, 0.361, 0.896)